In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import applications
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
from keras.applications.mobilenet import MobileNet
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.optimizers import RMSprop
from keras import backend as K
import numpy as np
import pandas as pd
from keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import load_model

Using TensorFlow backend.


In [2]:
#parameters
classes = 12
image_size = 1500
batch_size= 6
train_total = 3820
validation_total = 930

In [3]:
model_mobilenet = MobileNet(include_top=False, weights = None,input_shape=(image_size,image_size,3), alpha=0.5)

In [4]:
model_mobilenet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1500, 1500, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 750, 750, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 750, 750, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 750, 750, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 750, 750, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 750, 750, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 750, 750, 16)      0         
__________

In [5]:
x = model_mobilenet.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)

In [6]:
model = Model(inputs=model_mobilenet.input, outputs=x)

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1500, 1500, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 750, 750, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 750, 750, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 750, 750, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 750, 750, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 750, 750, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 750, 750, 16)      0         
__________

In [8]:
print(model.layers[1].trainable)

True


In [9]:
model.load_weights('mobilenet_from_scratch_image_1000_local_split.hdf5')

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   rotation_range = 45,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [11]:
train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 3820 images belonging to 12 classes.
Found 930 images belonging to 12 classes.


In [12]:
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-6),
             metrics=['accuracy'])

In [13]:
callbacks = [ModelCheckpoint(filepath='mobilenet_from_scratch_image_1500_local_split.hdf5', verbose=1, save_best_only=True, save_weights_only=True),
ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1),
EarlyStopping(monitor='val_loss', patience=20, verbose=1),
CSVLogger('./12-metrics_increase.csv')]

In [100]:
history = model.fit_generator(
                    train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2,
                    callbacks=callbacks,
                    initial_epoch=24)

Epoch 25/100
Epoch 00025: val_loss improved from inf to 0.08861, saving model to mobilenet_from_scratch_image_1500_local_split.hdf5
 - 1848s - loss: 0.0737 - acc: 0.9767 - val_loss: 0.0886 - val_acc: 0.9774
Epoch 26/100
Epoch 00026: val_loss improved from 0.08861 to 0.08843, saving model to mobilenet_from_scratch_image_1500_local_split.hdf5
 - 1789s - loss: 0.0757 - acc: 0.9757 - val_loss: 0.0884 - val_acc: 0.9774
Epoch 27/100
Epoch 00027: val_loss improved from 0.08843 to 0.08713, saving model to mobilenet_from_scratch_image_1500_local_split.hdf5
 - 1813s - loss: 0.0709 - acc: 0.9806 - val_loss: 0.0871 - val_acc: 0.9742
Epoch 28/100
Epoch 00028: val_loss did not improve
 - 1807s - loss: 0.0705 - acc: 0.9757 - val_loss: 0.0874 - val_acc: 0.9763
Epoch 29/100


KeyboardInterrupt: 

In [14]:
model.load_weights('mobilenet_from_scratch_image_1500_local_split.hdf5')

In [15]:
model.evaluate_generator(validation_generator, int(np.ceil(validation_total/batch_size)))

[0.087134061432203674, 0.97419354607982023]

In [16]:
model.save('mobilenet_1500_local_split.hdf5')

In [14]:
model_2 = load_model('mobilenet_1500_local_split.hdf5')

ValueError: Unknown activation function:relu6

In [17]:
# validation predictions

In [13]:
mobilenet_1500_local_split.hdf5validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 930 images belonging to 12 classes.


In [14]:
predictions_valid = model.predict_generator(validation_generator,int(np.ceil(validation_total/batch_size)))

In [15]:
predictions_valid = np.argmax(predictions_valid, axis=1)


In [16]:
true_positive = np.zeros(12)
false_positive = np.zeros(12)
true_negative = np.zeros(12)
false_negative = np.zeros(12)
for i in range(len(predictions_valid)):
    if(predictions_valid[i]==validation_generator.classes[i]):
        true_positive[predictions_valid[i]] = true_positive[predictions_valid[i]] + 1
    else:
        false_positive[predictions_valid[i]] = false_positive[predictions_valid[i]] + 1
        false_negative[validation_generator.classes[i]] = false_negative[validation_generator.classes[i]] +1

In [17]:
print('true_positive')
print(true_positive)
print('false_positive')
print(false_positive)
print('false_negative')
print(false_negative)
print('Total')
print(true_positive+false_negative)

true_positive
[  40.   93.   64.  106.   44.   86.  125.   36.  106.   41.   95.   70.]
false_positive
[ 6.  0.  1.  2.  0.  1.  9.  0.  2.  2.  0.  1.]
false_negative
[ 7.  1.  1.  2.  1.  3.  6.  0.  1.  1.  0.  1.]
Total
[  47.   94.   65.  108.   45.   89.  131.   36.  107.   42.   95.   71.]


In [112]:
# test submission

In [16]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
                    'test',
                    shuffle=False,
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode=None)


Found 794 images belonging to 1 classes.


In [17]:
predictions = model.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes = [class_ids[x] for x in np.argmax(predictions, axis=1)]


In [18]:
from os.path import basename
test_ids = [basename(x) for x in test_generator.filenames]
submission = pd.DataFrame({'file':test_ids,'species':predicted_classes})
submission.to_csv('submission_mobilenet_size_increase_1500_local_split_only.csv', encoding="utf8", index=False)

In [19]:
from IPython.display import FileLink
FileLink('submission_mobilenet_size_increase_1500_local_split_only.csv')

/home/sainath/plant_seeding/submission_mobilenet_size_increase_1500_local_split_only.csv

In [ ]:
# PB LB: 0.97858